## PySMuRF Setup

In [1]:
import pysmurf.client
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import epics
import os
import itertools
import pickle
from os.path import exists
import signal

%matplotlib inline

Next we'll define the EPICS server prefix and experiment configuration file

In [2]:
tone_power = 13
epics_prefix = "smurf_server_s2"
config_file  = "/usr/local/src/pysmurf/cfg_files/mit/experiment_mit.cfg"
S = pysmurf.client.SmurfControl(epics_root=epics_prefix, cfg_file=config_file, setup=False, make_logfile=False)

In [3]:
S.set_fixed_tone(4300, 13, write_log=False)

(0, 168)

Now can talk to system using S object.

In [4]:
S.all_off()

[ 2022-11-29 22:20:06 ]  Turning off tones
[ 2022-11-29 22:20:14 ]  Turning off flux ramp
[ 2022-11-29 22:20:14 ]  Turning off all TES biases


Note the PSD frequency is plotted +/- 0MHz. Each digitizer has complex data at 614.4MHz corresponding to +/- 307.2MHz BW at baseband.  For band=3 this is 0MHz + 5750MHz = 4250MHz.


### Transfer function of the system

Next we can look at a transfer function estimate.  This will play a known random number sequence through the DAC and record the ADC response.  We'll use cross correlation to generate a transfer function estimate:

In [ ]:
# For the UC attenuators, the range you can supply is between 0-31 (an integer) where each integer step adds +0.5dB to the attenuation
S.set_att_uc(b=0,val=10) # Make sure upconvert attenuators which set the SMuRF output power are zero
f,resp=S.full_band_resp(band=0, make_plot=True, show_plot=True, n_scan=10)

#### The response is measured about 0MHz baseband which for band=0 is 4250 MHz RF.  We expect two resonators in band 0 ; one at 4.014 GHz and one at 4.388 GHz, or at  MHz and  MHz.

In [ ]:
f_res_arr = [4.013e9, 4.3872e9] # expected resonant frequencies
q_exp_arr = [4000,10000] # expected coupled quality factors

for f_res, q_exp in zip(f_res_arr, q_exp_arr):
    df = f_res/q_exp*2
    f_res_band = np.mod(f_res,500e6)-250e6
    plt.xlim((f_res_band-df),(f_res_band+df))
    plt.ylim(-15,5)
    plt.plot(f,10.*np.log10(np.abs(resp)))
    plt.show()

### Identifying resonator locations

Each SMuRF band is processed by an oversampled x2 polyphase filter bank.  The filter bank breaks the +/-307.2MHz band into subbands.  Subband 0...127 correspond to -307.2MHz...307.2MHz.  Only subbands +/-250MHz are processed.  We can do a tone sweep to see the subband structure and identify resonator locations within each subband.  The plot shows the interleaved subbands in different colors.

In [ ]:
S.find_freq(band=0, tone_power=tone_power, make_plot=True, show_plot=True)

### Fine resolution resonator scans
Next we run a fine resonator scan and set the nominal tone frequency, drive power and rotation.  Feedback operates on the RF demodulated Q and performs feedback to drive Q to 0.

In [ ]:
S.setup_notches(band=0, tone_power=tone_power, sweep_width=1, delta_freq=0.1, df_sweep=0.01, new_master_assignment=True)
# Plots will be saved in S.plot_dir
S.plot_tune_summary(band=0, eta_scan=True)
print("These plots are saved at: " + S.plot_dir)
# run a gradient descent to find resonator dip
S.run_serial_gradient_descent(0) 
# run eta scan to normalize resonator gain (delta Q -> delta frequency) and apply rotation such that Q is nominally 0
S.run_serial_eta_scan(0) 

In [ ]:
S.plot_dir

# Tracking on the resonators

First let's check if we see the resonators respond to the flux ramp at all.  For that you can disable tracking and turn the flux ramp on ; you should see the resonator modulate in the error plot.  

In [ ]:
S.flux_ramp_off()
S.set_feedback_enable(0,1)

In [ ]:
S.set_filter_disable(False)
S.set_downsample_factor(20)

In [ ]:
S.which_on(0)

In [ ]:
S.tracking_setup(band=0,channel=S.which_on(0),reset_rate_khz=4, fraction_full_scale=0.8,make_plot=True,show_plot=True, lms_freq_hz=None, meas_flux_ramp_amp=True, n_phi0=3, feedback_start_frac=0.05, feedback_end_frac=0.95, lms_gain=1)

# Let's try taking Noise

The conversion from rad in phi0 to pA is done using this number;

In [ ]:
S._pA_per_phi0

Here's how you take 15 seconds of noise;

In [ ]:
#S.take_noise_psd(meas_time=30, show_plot=True)

# Optimize over a couple of parameters.

In [ ]:
class TimeOutException(Exception):
    def __init__(self, message):
        super(TimeOutException, self).__init__(message)
        
def signal_handler(signum, frame):
    raise TimeOutException("Timeout!") 
    
signal.signal(signal.SIGALRM, signal_handler)
signal.alarm(200000)

In [ ]:
if exists(S.output_dir+"/noise_dict.pckl"):
    noise_dict = pickle.load( open( S.output_dir+"/noise_dict.pckl", "rb" ) )
    print("Noise dictionary loaded from memory")
else:
    noise_dict = {}

In [ ]:
param_list = []

band = 0
channel = 92
tone_powers_arr = [11,12,13,14]
att_arr = [0,2,4,6]
phin_arr = [2,3,4]
lms_gain_arr = [1,2,3,4]
ramp_freq_arr = [3,4,6,10]

combos = list(itertools.product(tone_powers_arr, att_arr,phin_arr,lms_gain_arr,ramp_freq_arr))

for i,combo in enumerate(combos):
    tone_power, att, phin, lms_gain,ramp_freq  = combo
    print(i, tone_power, att, phin, lms_gain, ramp_freq)
    
    if combo in noise_dict:
        print("Already performed")
        
    else:
        try:
            S.set_att_uc(band,att)
            S.setup_notches(band=band, 
                            tone_power=tone_power, 
                            sweep_width=1, 
                            delta_freq=0.1, 
                            df_sweep=0.002, 
                            new_master_assignment=True)
            S.plot_tune_summary(band=0, eta_scan=True)
            S.run_serial_gradient_descent(band)
            S.run_serial_eta_scan(band)
            S.set_feedback_enable(0,1)
            S.set_filter_disable(True)
            S.set_downsample_factor(1)
            f, df, sync = S.tracking_setup(band=band, 
                             channel=channel, 
                             reset_rate_khz=ramp_freq, 
                             fraction_full_scale=0.85, 
                             lms_freq_hz=None, 
                             meas_flux_ramp_amp=True, 
                             n_phi0=phin,
                             make_plot=True, 
                             lms_gain=lms_gain, 
                             feedback_start_frac=0.1, 
                             feedback_end_frac=1-0.1, 
                             lms_enable1=1, 
                             lms_enable2=1, 
                             lms_enable3=1)
            
            fileloc, params = S.take_noise_psd(channel=channel,
                                               meas_time=60,
                                               save_data=True,
                                               show_plot=True,
                                               make_channel_plot=True,
                                               make_summary_plot=False,
                                               return_noise_params=True,
                                               write_log=False)
            noise_dict[combo] = {}
            noise_dict[combo]['tracking'] = [f, df, sync]
            noise_dict[combo]['params'] = params
            plt.show()
            pickle.dump(noise_dict, open( S.output_dir+"/noise_dict.pckl", "wb" ) )
        except ValueError as err:
            print("Value error: {0}".format(err))
            S.all_off()
            S.flux_ramp_off()
            print(combo, "Failed, skipping!")
            noise_dict[combo] = None
        except TimeOutException as err: 
            print("Smurf died, restart manually, error: {0}".format(err))
            break
        #except ChannelAccessGetFailure:
            #print("Value error: {0}".format(err))
            #S.all_off()
            #print(combo, "Failed, skipping!")
            #noise_dict[combo] = None
            

In [ ]:
noise_dict

In [ ]:
pickle.dump(noise_dict, open( S.output_dir+"/noise_dict.pckl", "wb" ) )

In [ ]:
pas_list = []
for k,v in noise_dict.items():
    if v!=None:
        for i,res in enumerate(v['params'][0]):
            if abs(res-4387)<2:
                pas = v['params'][1][1][i]
    pas_list.append(pas)

In [ ]:
plt.hist(pas_list,bins=20)

In [ ]:
noise_dict